# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")
```

env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
print("Brain Name: ", brain_name)
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print("State sizes: ", states.shape)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

# main function that sets up environments
# perform training loop

import envs
from buffer import ReplayBuffer
from maddpg import MADDPG
import torch
import numpy as np
from tensorboardX import SummaryWriter
import os
from utilities import transpose_list, transpose_to_tensor

# keep training awake
from workspace_utils import keep_awake

# for saving gif
import imageio

def seeding(seed=1):
    np.random.seed(seed)
    torch.manual_seed(seed)

def pre_process(entity, batchsize):
    processed_entity = []
    for j in range(3):
        list = []
        for i in range(batchsize):
            b = entity[i][j]
            list.append(b)
        c = torch.Tensor(list)
        processed_entity.append(c)
    return processed_entity


def main():
    seeding()
    # number of parallel agents
    parallel_envs = 4
    # number of training episodes.
    # change this to higher number to experiment. say 30000.
    number_of_episodes = 1000
    episode_length = 80
    batchsize = 1000
    # how many episodes to save policy and gif
    save_interval = 1000
    t = 0
    
    # amplitude of OU noise
    # this slowly decreases to 0
    noise = 2
    noise_reduction = 0.9999

    # how many episodes before update
    episode_per_update = 2 * parallel_envs

    log_path = os.getcwd()+"/log"
    model_dir= os.getcwd()+"/model_dir"
    
    os.makedirs(model_dir, exist_ok=True)

    torch.set_num_threads(parallel_envs)
    env = envs.make_parallel_env(parallel_envs)
    
    # keep 5000 episodes worth of replay
    buffer = ReplayBuffer(int(5000*episode_length))
    
    # initialize policy and critic
    maddpg = MADDPG()
    logger = SummaryWriter(log_dir=log_path)
    agent0_reward = []
    agent1_reward = []
    agent2_reward = []

    # training loop
    # show progressbar
    import progressbar as pb
    widget = ['episode: ', pb.Counter(),'/',str(number_of_episodes),' ', 
              pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar(marker=pb.RotatingMarker()), ' ' ]
    
    timer = pb.ProgressBar(widgets=widget, maxval=number_of_episodes).start()

    # use keep_awake to keep workspace from disconnecting
    for episode in keep_awake(range(0, number_of_episodes, parallel_envs)):

        timer.update(episode)


        reward_this_episode = np.zeros((parallel_envs, 3))
        all_obs = env.reset() #
        # obs.shape = env_agent x N_agent x features
        obs, obs_full = transpose_list(all_obs)

        #for calculating rewards for this particular episode - addition of all time steps

        # save info or not
        save_info = ((episode) % save_interval < parallel_envs or episode==number_of_episodes-parallel_envs)
        frames = []
        tmax = 0
        
        if save_info:
            frames.append(env.render('rgb_array'))


        
        for episode_t in range(episode_length):

            t += parallel_envs
            

            # explore = only explore for a certain number of episodes
            # action input needs to be transposed
            actions = maddpg.act(transpose_to_tensor(obs), noise=noise)
            noise *= noise_reduction
            
            actions_array = torch.stack(actions).detach().numpy()

            # transpose the list of list
            # flip the first two indices
            # input to step requires the first index to correspond to number of parallel agents
            actions_for_env = np.rollaxis(actions_array,1)
            
            # step forward one frame
            next_obs, next_obs_full, rewards, dones, info = env.step(actions_for_env)
            
            # add data to buffer
            transition = (obs, obs_full, actions_for_env, rewards, next_obs, next_obs_full, dones)
            
            # Buffer will save [N_agent x features], No. of evn_agent = Number of samples push to buffer at once
            buffer.push(transition)
            
            reward_this_episode += rewards

            obs, obs_full = next_obs, next_obs_full
            
            # save gif frame
            if save_info:
                frames.append(env.render('rgb_array'))
                tmax+=1
        
        # update once after every episode_per_update
        if len(buffer) > batchsize and episode % episode_per_update < parallel_envs:
            for a_i in range(3):
                samples = buffer.sample(batchsize)
                maddpg.update(samples, a_i, logger)
            maddpg.update_targets() #soft update the target network towards the actual networks

        
        
        for i in range(parallel_envs):
            agent0_reward.append(reward_this_episode[i,0])
            agent1_reward.append(reward_this_episode[i,1])
            agent2_reward.append(reward_this_episode[i,2])

        if episode % 100 == 0 or episode == number_of_episodes-1:
            avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward), np.mean(agent2_reward)]
            agent0_reward = []
            agent1_reward = []
            agent2_reward = []
            for a_i, avg_rew in enumerate(avg_rewards):
                logger.add_scalar('agent%i/mean_episode_rewards' % a_i, avg_rew, episode)

        #saving model
        save_dict_list =[]
        if save_info:
            for i in range(3):

                save_dict = {'actor_params' : maddpg.maddpg_agent[i].actor.state_dict(),
                             'actor_optim_params': maddpg.maddpg_agent[i].actor_optimizer.state_dict(),
                             'critic_params' : maddpg.maddpg_agent[i].critic.state_dict(),
                             'critic_optim_params' : maddpg.maddpg_agent[i].critic_optimizer.state_dict()}
                save_dict_list.append(save_dict)

                torch.save(save_dict_list, 
                           os.path.join(model_dir, 'episode-{}.pt'.format(episode)))
                
            # save gif files
            imageio.mimsave(os.path.join(model_dir, 'episode-{}.gif'.format(episode)), 
                            frames, duration=.04)

    env.close()
    logger.close()
    timer.finish()

if __name__=='__main__':
    main()

### 4.1 Import settings
Initiate the environment

In [1]:
import os
import torch 
import random
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from unityagents import UnityEnvironment
import numpy as np
from maddpg_ import *
# !pip install progressbar2
import progressbar as pb
from torch.utils.tensorboard import SummaryWriter

#---------------Environment set-up--------------------#
env =UnityEnvironment(file_name='Tennis_Linux_NoVis/Tennis.x86_64')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Initiate environment
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0

# number of agents in the environment
print('Number of agents:', len(env_info.agents))
# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)
state_size = len(state)
print('States have length:', state_size)
#agent = DDPGAgent(state_size=state_size, action_size=action_size, seed=0)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Number of actions: 2
States have length: 24


ddpg_body_dim = [(256,128),(256,128)]

maddpg = MADDPG(state_size, action_size, ddpg_body_dim=ddpg_body_dim, seed=0, batch_size=128, buffer_size=int(10e5), tau=1e-3,\
                gamma=0.99, decay_noise=0.99,lr_actor=1e-3, lr_critic=1e-3, weight_decay=1e-6, device=device)

In [ ]:
def main():
    """
    Multiagent Deep deterministic Policy Gradient (MADDPG)
    This specific MMADDPG will learn how to play a game of tennis. There are 24 states-observables and 2 action-dimensions for
    each agent. 
        
    """
    # ---------------Configuration----------------#
    # Episodes and Trajectory
    n_episodes = 5000
    trajectory_length = 50
    batch_size = 128
    
    # Model Parameters
    ddpg_body_dim = [(400,128),(400,128)]
    seed = 0
    buffer_size = int(1e5)
    tau = 1e-3
    gamma = 0.99
    decay_noise = 0.99
    lr_actor = 1e-4
    lr_critic = 1e-4
    weight_decay = 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    print("Device used: ", device)
    maddpg = MADDPG(state_size, action_size, ddpg_body_dim=ddpg_body_dim, seed=seed,\
                    batch_size=batch_size, buffer_size=buffer_size, tau=tau, gamma=gamma,\
                    decay_noise=decay_noise,lr_actor=lr_actor, lr_critic=lr_critic,\
                    weight_decay=weight_decay, device=device)
    
    # Save configurations
    log_path = os.getcwd()+"/log"
    model_dir= os.getcwd()+"/model_dir"
    os.makedirs(model_dir, exist_ok=True)
    
    # initialize logging of rewards
    logger = SummaryWriter(log_dir=log_path)
    agent0_reward = []
    agent0_window = deque(maxlen=10)
    agent1_reward = []
    agent1_window = deque(maxlen=10)
    avg_rewards = 0
    avg_rewards_window = 0
    

    
    
    widget = ["Training Status: ", pb.Timer(), ' ', pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar()]
    #timer = pb.progressbar(widgets=widget, maxval=n_episodes).start()
    
    ##############################################################################
    #---------------------------Training Commence--------------------------------#
    for i_episode in pb.progressbar(range(1, n_episodes+1), widgets=widget):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        states = env_info.vector_observations           # get the current state
        score = np.zeros(len(env_info.agents))
        for t in range(trajectory_length):
        
            actions = maddpg.acts(states, noise=1.0)  # state[n_agent,dim], noise=1.0 (on)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations   # get the next state: array -  (num_agent, state_feature_dim)
            rewards = env_info.rewards                   # get the reward: list - [num_agent]
            dones = env_info.local_done                  # see if episode has finished: list - [num_agent]
            
            # Add to memory
            maddpg.memory.add_experience(states,actions,rewards, next_states,dones)
            
            score += rewards                               # update the score
            states = next_states                             # roll over the state to next time step
            if np.any(dones):
                break
            
            
            if len(maddpg.memory) > batch_size:
                for agent_i in range(2):
                    experiences = maddpg.memory.sample(batch_size)
                    maddpg.step(experiences, agent_i, logger)
                maddpg.update_targets()
        # Update once after every episode
        maddpg.tau*=0.99998                   # Reduce the learning of soft-update
           
        agent0_reward.append(score[0])       # save latest score into a variable containing the 100 latest scores
        agent0_window.append(score[0])     # save latest score
        agent1_reward.append(score[1])
        agent1_window.append(score[1])
        avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward)]
        avg_rewards_window = [np.mean(agent0_window), np.mean(agent1_window)]
        if i_episode % 100 ==0 or i_episode==n_episodes-1:
           
            agent0_reward = []
            agent1_reward = []
            for agent_i , (avg_reward, avg_reward_window) in enumerate(zip(avg_rewards, avg_rewards_window)):
                logger.add_scalar("Agent%i/mean_episode_rewards"% agent_i, avg_reward, avg_reward_window, i_episode)
            
        if i_episode % 100 == 0:
            print('\rEpisode {}\tScore: {}\t 10 Moving Average Score: {}, OUNOISE: {}'.format(i_episode, avg_rewards, avg_rewards_window, maddpg.maddpg_agent[0].noise.state*maddpg.maddpg_agent[0].decay_noise))
            
        # save model
        agent_checkpoints = []
        if np.mean(avg_rewards_window)>=0.5 and i_episode >=100:
            
            print('\nEnvironment solved in {:d} episodes!\t Score: {:5.2f}\t 10 Moving Average Score: {:.2f}'.format(i_episode, avg_rewards, avg_rewards_window))
            for agent_i in range(2):
                checkpoint = {
                    'net_params': maddpg.maddpg_agent[agent_i].network_local.state_dict()}   
                agent_checkpoints.append(checkpoint)
            print("\nModel saved!💎🙌")
            torch.save(agent_checkpoints, model_dir+'checkpoint.pt')
    logger.close()
    
    return (avg_rewards, avg_rewards_window,maddpg)
if __name__=="__main__":
    main()
output_tuple = main()

Device used:  cuda:0


Training Status: Elapsed Time: 0:00:01   0% ETA:   0:18:15 |                  |

-40789.20703125 12400.8076171875
-39447.78125 11886.1875
-40725.91796875 12098.240234375
-39272.671875 11553.5908203125
-40349.55859375 12442.18359375
-39605.9140625 11365.5546875
-40647.4765625 12505.34765625
-38842.7109375 11777.5908203125
-40511.6328125 12389.8505859375
-39335.75 11840.080078125
-40555.90625 12120.830078125
-39422.125 11871.591796875
-41396.5234375 12373.349609375
-39130.1015625 11709.259765625
-41572.21875 11770.203125
-39638.3359375 11472.587890625
-42144.53125 11622.4765625
-39914.87890625 10743.580078125
-41547.9765625 12076.6171875
-38497.83203125 11794.16015625


Training Status: Elapsed Time: 0:00:02   0% ETA:   0:21:01 |                  |

-41266.921875 11461.4931640625
-39451.25 11558.48828125
-40881.71875 12101.3916015625
-38968.875 11388.3818359375
-41778.43359375 10902.51953125
-39145.015625 11044.46875
-40910.6171875 12313.498046875
-39502.3515625 11833.484375
-42056.1875 12296.916015625
-40163.625 11115.3154296875
-41670.69921875 12072.259765625
-39561.11328125 11083.658203125
-42310.26171875 11833.916015625
-40767.62109375 11277.830078125
-42185.859375 12082.943359375
-40459.1640625 11925.2802734375
-42365.1015625 11690.310546875
-40390.3515625 11086.23046875
-42165.7421875 11950.74609375
-41508.83984375 10913.65625
-43319.578125 10484.01171875
-41408.0234375 10612.2822265625
-43121.7265625 11993.63671875
-39931.1328125 11101.50390625
-43299.97265625 11782.58203125
-41117.4296875 11604.0244140625
-44199.7578125 12092.93359375
-41090.19921875 11018.673828125
-42672.0859375 11708.59375
-42103.8515625 11587.130859375
-46586.16015625 11568.314453125
-42032.5078125 11595.166015625
-45918.828125 9521.677734375
-42025.87

...

-40533.17578125 10755.6943359375
-38723.0234375 10994.375
-46135.73046875 11619.154296875
-37266.9296875 8188.6591796875
-41704.078125 9392.30078125
-38506.7890625 9815.99609375
-43138.734375 11045.8935546875
-39685.4296875 8448.4453125
-40154.7265625 13340.2568359375
-37163.07421875 11776.4580078125
-40720.0703125 10976.9921875
-38894.734375 11913.77734375
-45304.73046875 8849.296875
-38396.2734375 10406.8896484375
-42573.07421875 7079.8046875
-38386.921875 9360.662109375
-43541.08984375 11311.62109375
-35222.2890625 9482.32421875
-40459.7265625 10645.98046875
-36138.98046875 9469.38671875
-39985.25 12246.970703125


Training Status: Elapsed Time: 0:00:21   0% ETA:   0:39:18 |                  |

-39476.4921875 8673.009765625
-40873.84375 11065.89453125
-36833.9140625 8412.6337890625
-38886.5859375 12290.96875
-37598.09765625 10920.81640625
-40958.30078125 8745.2080078125
-36764.3203125 9920.9248046875
-45573.34765625 10408.7177734375
-33895.03125 10613.015625
-42913.81640625 8324.017578125
-34192.4921875 11049.4970703125
-43704.7109375 11384.484375
-37956.30859375 10463.765625
-42379.265625 9287.9052734375
-34792.3984375 9837.04296875
-44888.765625 10285.9443359375
-37977.97265625 11201.66796875
-42390.28515625 8461.5751953125
-37732.90234375 9001.416015625
-43446.18359375 8197.59765625
-36879.01171875 9006.099609375
-42976.66796875 9650.033203125
-35961.5625 10874.853515625
-42592.0703125 8693.01171875
-36901.8515625 10274.849609375
-40773.9375 12869.794921875
-36407.3671875 8744.951171875
-38307.52734375 12871.5048828125
-38737.6953125 9285.625
-42587.25 11781.14453125
-39159.8359375 10304.8447265625


Training Status: Elapsed Time: 0:00:21   0% ETA:   0:39:23 |                  |

-43710.41015625 9077.138671875
-38204.8046875 10254.703125
-42715.359375 8638.822265625
-37829.7109375 9095.41015625
-40931.53125 10945.9443359375
-38884.265625 7009.5703125
-39920.578125 10683.3818359375
-38946.08984375 8907.9072265625
-42667.78125 8325.837890625
-37110.76953125 10767.9345703125
-40576.4375 12015.23828125
-38959.734375 7578.9482421875
-40895.33203125 11149.4423828125
-36249.5703125 8829.228515625
-40798.39453125 10979.2109375
-39751.6484375 10757.4970703125
-38041.91796875 10296.068359375
-38538.29296875 10180.7822265625
-41861.7734375 9935.69140625
-35272.33203125 8488.474609375
-41858.921875 8474.5595703125


In [ ]:
output_tuple[0]

In [ ]:
print("\rHi",end="")
print("\rRoger",end="")